In [2]:
import os

# takes care of annoying TF-GPU warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# remove useless Tensorflow warning:
# WARNING:absl:Found untraced functions such as _update_step_xla, lstm_cell_1_layer_call_fn, 
# lstm_cell_1_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, 
# lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). 
# These functions will not be directly callable after loading.
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [3]:
# very useful for managing wandb runs: https://stackoverflow.com/questions/71106179/log-two-model-runs-with-keras-wandb
import wandb
from wandb.keras import WandbCallback
os.environ["WANDB_SILENT"] = "true"

#### Logistic Regression: Formula Label Prediction (multi-label, all features)

In [4]:
import numpy as np 
import pandas as pd
from pathlib import Path 
import ast

import tensorflow as tf
import keras_tuner as kt

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import datetime

tfds.disable_progress_bar()
wandb_project_name = "multi_label_formula_classification"

In [5]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_"+metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_"+metric])

#### Import Data and Preprocess Dataframe

In [6]:
def preprocess_data(corpus,
                    irrelevant_features=["mtype",]):
    # drop irrelevant columns
    corpus.drop(irrelevant_features, inplace=True, axis=1)

    def cell_str_to_list(cell_val):
        return ast.literal_eval(cell_val)

    # filter strings
    def process_cell(cell_str):
        stripped_f_str = cell_str[1:-1].replace("\\\\", "\\")
        f_list = stripped_f_str.split(",")
        f_list = [token.replace("'", "").replace(" ", "") for token in f_list]
        f_list = ["{" if token == "\\{" else token for token in f_list]
        f_list = ["}" if token == "\\}" else token for token in f_list]
        cell_str = " ".join(f_list)
        return cell_str

    corpus["type_tokens"] = corpus["type_tokens"].map(process_cell)
    corpus["tokens"] = corpus["tokens"].map(process_cell)
    corpus["mtype_one_hot"] = corpus["mtype_one_hot"].map(cell_str_to_list)
    corpus["labels"] = corpus["labels"].map(cell_str_to_list)
    corpus = corpus.loc[(corpus["tokens"].str.len() > 0) & (corpus["tokens"] != " ")]


In [7]:
#print(os.getcwd())
data_p = Path("../data/") / "multi_class_unbalanced_data_TOKENIZED_V1.csv"
data = pd.read_csv(data_p)
preprocess_data(data)
data.head()

,tokens,type_tokens,labels,labels_str,mtype_one_hot
0,f,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
1,j : \mathbb{N} \rightarrow [ 0 1 ],func_def func_name __ANON_1 COLON mapping set_...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
2,b,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
3,mathbb{Q,set_constant SET_BASIC,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[1, 0, 0]"
4,f,func_name __ANON_1,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['functions', 'elementary-set-theory']","[0, 0, 1]"


In [8]:
print(data["type_tokens"].map(lambda x: len((x.split(" ")))).max())
print(data["tokens"].map(lambda x: len((x.split(" ")))).max())

254
252


In [9]:
# ordinary datasets
SMALL_TRAIN_SIZE = 24620 - 2460
SMALL_TEST_SIZE = 2460
LARGE_TRAIN_SIZE = 106523 - 10650
LARGE_TEST_SIZE = 10650
# compact datasets
NUM_CLASSES = 40

labels_array = np.array(data["labels"].to_list())
m_type_array = np.array(data["mtype_one_hot"].to_list())
dat_as_ds = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"],m_type_array), name="data")
labels_ds = tf.data.Dataset.from_tensor_slices(labels_array, name="label")
data_as_ds = tf.data.Dataset.zip((dat_as_ds, labels_ds))

test_dataset = data_as_ds.take(SMALL_TEST_SIZE)
train_dataset = data_as_ds.skip(SMALL_TEST_SIZE)
train_dataset_unb = train_dataset

In [10]:
x_test = dat_as_ds.take(SMALL_TEST_SIZE)
y_test = labels_ds.take(SMALL_TEST_SIZE)

x_data = dat_as_ds.skip(SMALL_TEST_SIZE)
y_data = labels_ds.skip(SMALL_TEST_SIZE)


In [11]:
data_as_ds.element_spec

((TensorSpec(shape=(), dtype=tf.string, name=None),
  TensorSpec(shape=(), dtype=tf.string, name=None),
  TensorSpec(shape=(3,), dtype=tf.int64, name=None)),
 TensorSpec(shape=(40,), dtype=tf.int64, name=None))

##### Setup and Data Preparation

In [12]:
for (example_token, example_type, example_m_type), label in train_dataset.take(5):
    print("text: ", example_token.numpy())
    print("type: ", example_type.numpy())
    print("m_type: ", example_m_type.numpy())
    print("label: ", label.numpy())

text:  b'{ 2   4 }'
type:  b'explset L_BRACE_LITERAL set_enumeration item __ANON_3 COMMA item __ANON_3 R_BRACE_LITERAL'
m_type:  [1 0 0]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : I \\to X'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f ( a ) = 1'
type:  b'func_expr func_name_arg func_name __ANON_1 L_PAREN __ANON_1 R_PAREN EQUAL expr_atom __ANON_3'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'x'
type:  b'func_name __ANON_1'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : X \\to Y'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [13]:
BUFFER_SIZE = 2000
BATCH_SIZE = 64
STEPS_PER_EPOCH = np.floor(SMALL_TRAIN_SIZE/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.floor(SMALL_TEST_SIZE/BATCH_SIZE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

##### Text Encoding

In [14]:
# for int encoder
TYPE_TOKENS_MAX_SEQ_LEN = 260
TOKENS_MAX_SEQ_LEN = 260


# for other encoders 
TYPE_TOKENS_PAD_TO_MAX_TOKENS = 80
TOKENS_PAD_TO_MAX_TOKENS = 200
BIGRAM_PAD_TO_MAX_TOKENS = 350

In [15]:
def create_encoder(output_mode_str, n_grams, mode):
    if output_mode_str == "int":
        VOCAB_SIZE = 200
        if mode == "token":
            max_seq_len = TOKENS_MAX_SEQ_LEN
        elif mode == "type":
            max_seq_len = TYPE_TOKENS_MAX_SEQ_LEN

        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            output_sequence_length = max_seq_len,
            split="whitespace",
            max_tokens=VOCAB_SIZE)
        #TODO: adapt for different inputs
        if mode == "token": 
            encoder.adapt(train_dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text
        elif mode == "type":
            encoder.adapt(train_dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text
        return encoder
    
    if output_mode_str == "count" and n_grams == 2:
        max_seq_len = BIGRAM_PAD_TO_MAX_TOKENS
        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            pad_to_max_tokens = max_seq_len,
            split="whitespace",
            max_tokens=max_seq_len)
        
        #TODO: adapt for different inputs
        if mode == "token": 
            encoder.adapt(train_dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text
        elif mode == "type":
            encoder.adapt(train_dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text
        return encoder
    
    if mode == "token":
        max_seq_len = TOKENS_PAD_TO_MAX_TOKENS
    elif mode == "type":
        max_seq_len = TYPE_TOKENS_PAD_TO_MAX_TOKENS

    encoder = tf.keras.layers.TextVectorization(
        standardize=None,
        output_mode=output_mode_str,
        ngrams = n_grams,
        pad_to_max_tokens = max_seq_len,
        split="whitespace",
        max_tokens=max_seq_len)
    #TODO: adapt for different inputs
    if mode == "token": 
        encoder.adapt(train_dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text
    elif mode == "type":
        encoder.adapt(train_dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text
    
    return encoder


    

##### Representation 1: Use integer indices encoding

In [16]:
encoder_int_tokens = create_encoder("int", None, "token")
encoder_int_types = create_encoder("int", None, "type")

vocab_tokens = np.array(encoder_int_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_int_tokens.get_vocabulary())
vocab_types = np.array(encoder_int_types.get_vocabulary())
vocab_size_types = len(encoder_int_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
tokens (voc size):  128
types (voc size):  61


In [17]:
encoded_example_token = encoder_int_tokens(example_token).numpy()
encoded_example_types = encoder_int_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[ 2 11 26 14 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
(260,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[15  5  3 13 14  6 16  6  0  0  

##### Representation 2: Count Vectorizer

In [18]:
encoder_count_tokens = create_encoder("count", None, "token")
encoder_count_types = create_encoder("count", None, "type")

vocab_tokens = np.array(encoder_count_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_count_tokens.get_vocabulary())
vocab_types = np.array(encoder_count_types.get_vocabulary())
vocab_size_types = len(encoder_count_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

tokens (voc size):  127
types (voc size):  60


In [19]:
encoded_example_token = encoder_count_tokens(example_token).numpy()
encoded_example_types = encoder_count_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
(200,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[0. 0. 1. 0. 1. 2. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

##### Model (Logistic Regression)

 **Define and compile model**

In [20]:
def create_model(emb_dim, dp1, dp2, tokens_encoder, types_encoder, tokens_size, types_size, input_type, add_inp_emb_dim=1):
    embedding_input_dim = tokens_size + types_size + add_inp_emb_dim + 3 # 3 size of sem_type_model_input

    tokens_model_input = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
    tokens_vectorized = tokens_encoder(tokens_model_input)

    types_model_input = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
    types_vectorized = types_encoder(types_model_input)

    if input_type == "float":
        sem_type_model_input = tf.keras.layers.Input(dtype=tf.float32, shape=(3,))
    elif input_type == "int":
        sem_type_model_input = tf.keras.layers.Input(dtype=tf.int64, shape=(3,))

    merged = tf.keras.layers.Concatenate(axis=1)([tokens_vectorized, types_vectorized, sem_type_model_input])
    embedded = tf.keras.layers.Embedding(input_dim= embedding_input_dim,
                                        output_dim=emb_dim,
                                        # user masking to handle the variable sequence lengths
                                        mask_zero=True)(merged)
    
    dropout1 = tf.keras.layers.Dropout(dp1)(embedded)
    globalmaxpooling = tf.keras.layers.GlobalMaxPooling1D()(dropout1)
    dropout2 = tf.keras.layers.Dropout(dp2)(globalmaxpooling)
    model_output = tf.keras.layers.Dense(40)(dropout2)

    model = tf.keras.models.Model(inputs=[tokens_model_input, types_model_input, sem_type_model_input], outputs=model_output)
        
    return model
    

In [21]:
def create_model_builder(tokens_encoder, types_encoder, tokens_size, types_size, input_type, add_inp_emb_dim=1):

    def model_builder(hp):
        ...
        # model = create_model(encoder_count_tokens, encoder_count_types,tokens_input_len, "float", type_input_len)
        embedding_input_dim = tokens_size + types_size + add_inp_emb_dim + 3 # 3 size of sem_type_model_input

        tokens_model_input = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
        tokens_vectorized = tokens_encoder(tokens_model_input)

        types_model_input = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
        types_vectorized = types_encoder(types_model_input)

        if input_type == "float":
            sem_type_model_input = tf.keras.layers.Input(dtype=tf.float32, shape=(3,))
        elif input_type == "int":
            sem_type_model_input = tf.keras.layers.Input(dtype=tf.int64, shape=(3,))

        merged = tf.keras.layers.Concatenate(axis=1)([tokens_vectorized, types_vectorized, sem_type_model_input])

        hp_emb_dims = hp.Int('emb_dims', min_value=32, max_value=128, step=32)
        embedded = tf.keras.layers.Embedding(input_dim= embedding_input_dim,
                                            output_dim=hp_emb_dims,
                                            # user masking to handle the variable sequence lengths
                                            mask_zero=True)(merged)
    
        hp_do1 = hp.Choice('dropout1', values=[0.1, 0.2, 0.3])
        dropout1 = tf.keras.layers.Dropout(hp_do1)(embedded)
        globalmaxpooling = tf.keras.layers.GlobalMaxPooling1D()(dropout1)

        hp_do2 = hp.Choice('dropout2', values=[0.1, 0.2, 0.3])
        dropout2 = tf.keras.layers.Dropout(hp_do2)(globalmaxpooling)
        model_output = tf.keras.layers.Dense(40)(dropout2)

        model = tf.keras.models.Model(inputs=[tokens_model_input, types_model_input, sem_type_model_input], outputs=model_output)
        
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(hp_learning_rate),
              metrics=["accuracy", tf.keras.metrics.Recall()])
        return model
    
    return model_builder


 **Train the model**

##### Experiment 1: Use integer indices for encoding tokens

##### 1.1 Determine best hyperparameters

In [22]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN

model_builder = create_model_builder(encoder_int_tokens, encoder_int_types,tokens_input_len, type_input_len, "int")

data_train = dat_as_ds
label_train = labels_ds

In [23]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_recall", direction="max"),
                     max_epochs=35,
                     factor=3,
                     directory='my_dir',
                     project_name='multi_label_small')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


INFO:tensorflow:Reloading Tuner from my_dir/multi_label_small/tuner0.json


In [24]:
tuner.search(train_dataset,
             epochs=30,
             validation_data=test_dataset,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

INFO:tensorflow:Oracle triggered exit


In [25]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.001
64
0.1
0.1


##### 1.2 Train with best hyperparameters

In [26]:

NUM_EPOCHS = 80
model1 = create_model(optimal_emb_dims, dp1, dp2, encoder_int_tokens, encoder_int_types, tokens_input_len, type_input_len, "int")
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [27]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "small-ordinary-unbalanced-all-inputs",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1.fit(train_dataset, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
343/346 [============================>.] - ETA: 0s - loss: 0.2859 - accuracy: 0.5326 - recall: 0.1115INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


346/346 [==============================] - 7s 20ms/step - loss: 0.2848 - accuracy: 0.5309 - recall: 0.1105 - val_loss: 0.1856 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/80
343/346 [============================>.] - ETA: 0s - loss: 0.1448 - accuracy: 0.4965 - recall: 0.1087INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


346/346 [==============================] - 7s 20ms/step - loss: 0.1448 - accuracy: 0.4949 - recall: 0.1084 - val_loss: 0.1756 - val_accuracy: 0.0000e+00 - val_recall: 2.2548e-04
Epoch 3/80
343/346 [============================>.] - ETA: 0s - loss: 0.1410 - accuracy: 0.5173 - recall: 0.1246INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


...

346/346 [==============================] - 6s 19ms/step - loss: 0.1279 - accuracy: 0.6512 - recall: 0.1865 - val_loss: 0.1350 - val_accuracy: 0.7899 - val_recall: 0.2047
Epoch 75/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1277 - accuracy: 0.6544 - recall: 0.1880 - val_loss: 0.1354 - val_accuracy: 0.7895 - val_recall: 0.2038
Epoch 76/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1278 - accuracy: 0.6560 - recall: 0.1908 - val_loss: 0.1359 - val_accuracy: 0.7895 - val_recall: 0.2056
Epoch 77/80
346/346 [==============================] - ETA: 0s - loss: 0.1278 - accuracy: 0.6582 - recall: 0.1925INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


346/346 [==============================] - 6s 18ms/step - loss: 0.1278 - accuracy: 0.6582 - recall: 0.1925 - val_loss: 0.1350 - val_accuracy: 0.7911 - val_recall: 0.2054
Epoch 78/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1275 - accuracy: 0.6582 - recall: 0.1960 - val_loss: 0.1354 - val_accuracy: 0.7895 - val_recall: 0.2101
Epoch 79/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1274 - accuracy: 0.6614 - recall: 0.1982 - val_loss: 0.1351 - val_accuracy: 0.7895 - val_recall: 0.2097
Epoch 80/80
346/346 [==============================] - ETA: 0s - loss: 0.1274 - accuracy: 0.6600 - recall: 0.2013INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_020001-vvuppqsn/files/model-best/assets


346/346 [==============================] - 7s 19ms/step - loss: 0.1274 - accuracy: 0.6600 - recall: 0.2013 - val_loss: 0.1347 - val_accuracy: 0.7911 - val_recall: 0.2108


In [43]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 260)         0           ['input_1[0][0]']                
 ization)                                                                                         
                                                                                                  
 text_vectorization_1 (TextVect  (None, 260)         0           ['input_2[0][0]']            

In [46]:
import io

print("tokens len: ", tokens_input_len)
print("types len: ", type_input_len)
try:
    weights = model1.get_layer('embedding').get_weights()[0]
    vocab1 = encoder_int_tokens.get_vocabulary()
    vocab2 = encoder_int_types.get_vocabulary()
    out_v = io.open('vectors1.tsv', 'w', encoding='utf-8')
    out_m = io.open('metadata1.tsv', 'w', encoding='utf-8')
    
    first_index = 0
    for index, word in enumerate(vocab1):
        if index == 0:
            continue  # skip 0, it's padding.
        vec = weights[index]
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        out_m.write(word + "\n")
        first_index = index
    first_index = tokens_input_len
    print(first_index)
    # tokens_input_len, type_input_len

    second_index = 0
    for index, word in enumerate(vocab2):
        if index == 0:
            continue  # skip 0, it's padding.
        second_index = index + first_index
        vec = weights[second_index]
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        out_m.write(word + "\n")

    second_index = tokens_input_len + type_input_len
    print(second_index)

    #set , scal , func ,
    for index, word in [(0, "SET"), (1, "SCAL"), (2, "FUNC")]:
        vec = weights[second_index + index]
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
        out_m.write(word + "\n")
    
    out_v.close()
    out_m.close()

except Exception as e:
    print(e)

200
280


##### Experiment 2:


##### 2.1 Determine best hyperparameters

In [29]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN

model_builder2 = create_model_builder(encoder_count_tokens, encoder_count_types,tokens_input_len, type_input_len, "float")


In [31]:
tuner2 = kt.Hyperband(model_builder2,
                     objective=kt.Objective("val_recall", direction="max"),
                     max_epochs=35,
                     factor=3,
                     directory='my_dir2',
                     project_name='multi_label_count_small')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [32]:
tuner2.search(train_dataset,
             epochs=30,
             validation_data=test_dataset,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 90 Complete [00h 00m 30s]
val_recall: 0.0

Best val_recall So Far: 0.0004509582940954715
Total elapsed time: 00h 37m 10s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


##### 2.2 Train with best hyperparameters

In [33]:
best_hps=tuner2.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.001
128
0.1
0.2


In [34]:
tokens_input_len = TOKENS_PAD_TO_MAX_TOKENS 
type_input_len = TYPE_TOKENS_PAD_TO_MAX_TOKENS

model2 = create_model(optimal_emb_dims, dp1, dp2, encoder_count_tokens, encoder_count_types,tokens_input_len, type_input_len, "float") # maybe add 30
# predict on a sample formula using untrained model
#predictions = model.predict(np.array([sample_text]))
#print(predictions[0])

In [35]:
NUM_EPOCHS = 80
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [36]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "small-ordinary-unbalanced-all-inputs",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}
run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2.fit(train_dataset, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback(),])#[tensorboard_callback])
run.finish()

Epoch 1/80
346/346 [==============================] - ETA: 0s - loss: 0.2544 - accuracy: 0.5737 - recall_1: 0.1627INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


346/346 [==============================] - 7s 19ms/step - loss: 0.2544 - accuracy: 0.5737 - recall_1: 0.1627 - val_loss: 0.1944 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/80
346/346 [==============================] - ETA: 0s - loss: 0.1455 - accuracy: 0.5068 - recall_1: 0.1321INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


346/346 [==============================] - 6s 18ms/step - loss: 0.1455 - accuracy: 0.5068 - recall_1: 0.1321 - val_loss: 0.1833 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/80
343/346 [============================>.] - ETA: 0s - loss: 0.1446 - accuracy: 0.5025 - recall_1: 0.1171INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


...

346/346 [==============================] - 6s 18ms/step - loss: 0.1461 - accuracy: 0.4081 - recall_1: 0.0099 - val_loss: 0.1574 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 45/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1459 - accuracy: 0.4114 - recall_1: 0.0105 - val_loss: 0.1604 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 46/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1463 - accuracy: 0.4030 - recall_1: 0.0104 - val_loss: 0.1585 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 47/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1462 - accuracy: 0.4113 - recall_1: 0.0100 - val_loss: 0.1602 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 48/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1460 - accuracy: 0.4163 - recall_1: 0.0105 - val_loss: 0.1584 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 49/80
346/346 [=====================

INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


346/346 [==============================] - 6s 19ms/step - loss: 0.1461 - accuracy: 0.4044 - recall_1: 0.0107 - val_loss: 0.1572 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 51/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1458 - accuracy: 0.4179 - recall_1: 0.0100 - val_loss: 0.1588 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 52/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1458 - accuracy: 0.4131 - recall_1: 0.0101 - val_loss: 0.1585 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 53/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1459 - accuracy: 0.4136 - recall_1: 0.0102 - val_loss: 0.1575 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 54/80
344/346 [============================>.] - ETA: 0s - loss: 0.1457 - accuracy: 0.4266 - recall_1: 0.0094INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-202303

INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_091057-m90ovok7/files/model-best/assets


346/346 [==============================] - 7s 20ms/step - loss: 0.1457 - accuracy: 0.4261 - recall_1: 0.0094 - val_loss: 0.1568 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 55/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1455 - accuracy: 0.4227 - recall_1: 0.0100 - val_loss: 0.1595 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 56/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1455 - accuracy: 0.4189 - recall_1: 0.0099 - val_loss: 0.1587 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 57/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1454 - accuracy: 0.4287 - recall_1: 0.0101 - val_loss: 0.1581 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 58/80
346/346 [==============================] - 5s 15ms/step - loss: 0.1451 - accuracy: 0.4408 - recall_1: 0.0107 - val_loss: 0.1582 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 59/80
346/346 [=====================